In [ ]:
import json
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from googletrans import Translator
from langdetect import detect




In [ ]:
df = pd.read_csv("Dataframe_Wrangled_NoDuplicates.csv")
df.head()

In [ ]:
print(len(df))

In [ ]:
# parse conversations
text_list = []
text_arrays = []
for conversation in df["conversations"]:
    text = ""
    text_arr = []
    parsed_conversation = eval(conversation)
    for dialogue in parsed_conversation:
        text += dialogue["value"] + " "
        text_arr.append(dialogue["value"])
    text_list.append(text)
    text_arrays.append(text_arr)

# create column with the conversation text
df["text"] = text_list
df["values"] = text_arrays
        


In [ ]:
df.head()


In [ ]:
# determine language of each conversation
languages = []
for value in df["values"]:
    try:
        lang = detect(value[0])
        languages.append(lang)
    except:
        try:
            lang = detect(value[1])
            languages.append(lang)
        except:
            print("error2")
            print(value[1])


df["lang"] = languages


In [ ]:
df.head()

In [ ]:
df_shortened = df[['id', 'values', 'lang']]
df_shortened.to_csv('file_name.csv', index=False, encoding='utf-8')


In [28]:
df["lang"].value_counts()

lang
en       18345
ko         822
zh-cn      624
fr         522
es         464
ja         238
ca         198
it         177
vi         163
pt         162
de         148
nl         133
no         133
ru         124
ro         121
sw          86
et          76
zh-tw       75
id          63
af          63
cy          62
fi          62
pl          56
da          54
tl          40
sv          33
hr          32
so          30
tr          29
sk          21
sl          21
uk          18
th          16
cs          11
lt          11
bg          10
he           9
hu           8
mk           6
sq           3
fa           3
lv           2
el           2
ar           1
Name: count, dtype: int64

In [ ]:
translator = Translator()

async def translate(value):
      result = await translator.translate(value, dest="en")
      return result

translated_values = []
for index, item in df["values"].items():
    if df["lang"][index] != "en":
        translated_conversation = []
        for element in item:
            a=re.compile("<.*?>")
            element = a.sub('', element)
            if len(element) > 15000:
                chunks = []
                for i in range(0, len(element), 15000):
                    chunks.append(element[i:i + 15000])
                
                translated_chunks = []
                for chunk in chunks:
                    result = await translate(chunk)
                    translated_chunks.append(result.text)
                
                result = " ".join(translated_chunks)
                translated_conversation.append(result)

            else:
                result = await translate(element)
                translated_conversation.append(result.text)
                
        translated_values.append(translated_conversation)
    else:
        translated_values.append(item)

df["translated_values"] = translated_values


# 67 conversations exceed 15,000


In [ ]:
corpus = df["text"]

In [ ]:
def preprocess_text(text):
    text = str(text)
    # remove angle brackets and all characters inside angle bracket
    # a=re.compile("<.*?>")
    # text = a.sub('', text)
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # convert to lowercase
    text = text.lower()
    # tokenize the text
    tokens = word_tokenize(text)
    # remove stopwords
    # tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

# preprocess all text
preprocessed_text = [' '.join(preprocess_text(document)) for document in corpus]

In [ ]:
df["preprocessed_text"] = preprocessed_text

In [ ]:
df.head()

In [ ]:
corpus = df["preprocessed_text"]

In [ ]:
# tokenize the documents
tokenized_documents = [doc.split() for doc in corpus]


In [ ]:
# create a dictionary representation of the documents
dictionary = Dictionary(tokenized_documents)

In [ ]:
# convert document into bag-of-words format
corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]

In [ ]:
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, random_state=42)


In [ ]:
topics = lda_model.print_topics()
for topic in topics:
    print(topic)